<a href="https://colab.research.google.com/github/Jeromy0515/Jeromy0515/blob/main/decision_tree/decision_tree5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 시작하기 전에
1. `max_depth`를 3 말고 다른 값으로 하면 성능은?
2. 지금까지 훈련 세트에서 모델을 훈련 -> 테스트 세트에서 모델 평가
3. 테스트 세트 점수를 보고, 이 모델을 실전 투했을 때의 성능을 기대 -> 일반화 성능 가늠
4. 테스트 세트를 사용해 성능을 확인하다 보면 점점 테스트 세트에 맞추게 되는셈


# 검증 세트
- 테스트 세트는 모델을 만들고 나서 마지막에 딱 한 번만 사용
- 테스트 세트를 사용하지 않으면 모델이 과대적합인지 과소적합인지 판단할 수 없음
- 테스트 세트를 사용하지 않고 이를 측정: 훈련 세트를 또 나눔 -> 검증세트(validation set)

In [1]:
# 데이터 로드
import pandas as pd
wine = pd.read_csv('/content/drive/MyDrive/Data/wine.csv')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [2]:
# class 열을 타겟으로 사용
# 나머지 열은 특성 배열에 저장
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
# 훈련 세트와 테스트 세트 나누기 8:2
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [4]:
# 검증 세트 생성
# train_input과 train_target을 다시 train_test_split()
# 훈련 세트 sub_input, sub_target
# 검증 세트 val_input, val_target
# train_input의 약 20%를 val_input
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)
print(sub_input.shape, val_input.shape)
# 원래 5,197개였던 훈련 세트가 4,157개로 줄었음 
# 검증 세ㅐ트는 1,040개

(4157, 3) (1040, 3)


In [5]:
# sub_input, sub_target과 val_input, val_target을 사용해 모델 생성 및 평가
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))
# 과대적합!
# 매개변수를 바꿔서 더 좋은 모델을 찾아야함

0.9971133028626413
0.864423076923077


# 교차 검증
- 검증 세트를 만드느라 훈련 세트가 줄었음
- 많은 데이터를 훈련에 사용할수록 좋은 모델 생성 가능
- 검증 세트를 너무 조금 떼어 놓으면 검증 점수가 들쭉날쭉 불안정
- 교차 검증(cross validation) 사용
  - 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터 사용 가능
  - 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복
  - 그 다음 이 점수를 평균하여 최종 검증 점수를 얻음

# k-폴드 교차 검증(k-fold cross validation)
- k-폴드 교차 검증(k-fold cross validation)
  - 훈련 세트를 몇 부분으로 나누냐에 따라 다르게 부름
  - 보통 5-폴드, 10-폴드 교차 검증을 많이 사용함
  - 데이터의 8~90% 까지 훈련에 사용: 안정된 점수


# cross_validate()
- 사이킷런에서는 cross_validate()라는 교차 검증 함수 제공
- 수작업으로 직접 검증 세트를 떼어내지 않고 훈련 세트 전체를 전달
- 기본적으로 5-폴드 교차 검증 수행
- `cv` 매개변수에서 폴드 수 변경 가능
- `fit_time`: 각각 모델을 훈련하는 시간, 각 키마다 5개의 숫자
- `score_time`: 각각 모델을 검증하는 시간, 각 키마다 5개의 숫자
- `test_score`: 각 폴드에서 계산한 검증 점수
- 교차 검증의 최종 점수: test_score 키에 담긴 5개의 점수를 평균

In [6]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)
# fit_time: 각각 모델을 훈련하는 시간, 각 키마다 5개의 숫자
# score_time: 각각 모델을 검증하는 시간, 각 키마다 5개의 숫자
# test_score: 각 폴드에서 계산한 검증 점수

{'fit_time': array([0.01738834, 0.00717902, 0.01134777, 0.00943303, 0.00722313]), 'score_time': array([0.00084734, 0.00075054, 0.00122499, 0.00086498, 0.0008471 ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [7]:
# 교차 검증의 최종 점수: test_score 키에 담긴 5개의 점수 평균 출력
import numpy as np
print(np.mean(scores['test_score']))
# train+test_split() 함수로 전체 데이터를 섞은 후 훈련 세트를 준비했기 때문에 -> 따로 폴드를 섞을 필요는 없음

0.855300214703487


In [9]:
# 회귀 모델: KFold 분할기 사용
# 분류 모델: StratifiedKFold 사용( 타깃 클래스를 골고루 나누기 위해 )
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [10]:
# 미리 수작업으로 폴드를 나누지 않아도 결과는 동일!
# n_splits 매개변수: 몇(k) 폴드 교차 검증할지 결정
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


# 하이퍼파라미터 튜닝
- 결정 트리의 매개변수 값을 바꿔가며 가장 좋은 성능이 나오는 모델을 찾자
- 테스트 세트가 아닌 교차 검증을 통해서 좋은 모델 찾기
- 모델 파라미터: 머신러닝 모델이 학습하는 파라미터
- 하이퍼파라미터: 모델이 학습할 수 없어서 사용자가 지정하는 파라미터
- 하이퍼파라미터를 튜닝하는 작업?

  1. 라이브러리가 제공하는 기본값 사용
  2. 검증 세트 점ㅈ수나 교차 검증을 통해 매개변수 변경
  3. 모델마다 1~6개의 매개변수 제공 -> 이를 다 바꿔가면서?
  4. `max_depth` 최적값이 변경하면 min_sample_split도 바뀜 -> 다 찾아야됨

#### <b>사이키런의 GridSearchCV 클래스를 통해 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행할 수 있음</b>

# GridSearchCV
- 하이퍼파라미터 탐색과 교차 검증을 한번에 수행
- 별도로 cross_validate() 함수 호출 필요 없음
- 기본 매개변수를 사용한 결정 트리 모델에서 `min_impurity_decrease` 매개변수의 최적값 찾기


In [11]:
# GridSearchCV 클래스 임포트
from sklearn.model_selection import GridSearchCV
# 탐색할 값의 리스트를 딕셔너리로 만듦(0.0001 ~ 0.0005: 0.001씩 증가)
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.004, 0.0005]}

In [12]:
# 결정 트리 클래스를 생성하자마자 바로 GridSearchCV로 전달
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
# fit() 메소드 호출: mean_impurity_decrease 값을 바꿔가며 총 5번 실행
# GridSearchCV의 cv 매개변수 기본값은 5
# min_impurity_decrese 값마다 5-폴드 교차 검증 수행: 5 x 5 = 25개의 모델 훈련
# n_jobs 매개변수: 병렬 실행에 사용할 CPU 코어 수(-1: 모든 코어 사용)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.004, 0.0005]})

- 교차 검증에서는 최적의 하이퍼차라미터를 찾으면 -> 전체 훈련 세트로 모델을 다시 만들어야 했음
- 그리드 서치는 훈련이 끄탄면 25개 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 자동으로 다시 모델을 훈련
- 이 모델은 gs 객체의 `best_estimator_` 속성에 저장 -> 일반 결정 트리처럼 똑같이 사용

In [14]:
# 최적의 모델은 gs 객체의 best_estimator_ 속성에 저장 -> 일반 결정 트리처럼 똑같이 사용
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


- 교차 검증에서는 최적의 하이퍼파라미터를 찾으면 -> 전체 훈련 세트로 모델을 다시 만들어야 했음
- 그리드 서치는 훈련이 끝나면 25개 모델 중에서 검증 점수가 가장 높은 모델의 배개변수 조합으로 자동으로 다시 모델을 훈련

In [15]:
# 최적의 매개변수는 best_params_ 속성에 저장
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [16]:
# 각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_ 속성의 'mean_test_score' 키에 저장
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.85414803 0.86761605]


- `min_impurity_decrease`: 노드를 분할하기 위한 불순도 감소 최소량 지정
- `max_depth`: 트리의 깊이 제한
- `min_samples_split`: 노드를 나누기 위한 최소 샘플 수 지정


In [17]:
# min_impurity_decrease: 9개
# max_depth: 15개
# min_samples_split: 10개
params = {
    'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
    'max_depth': range(5, 20, 1),
    'min_samples_split': range(2, 100, 10)
}

In [18]:
# 매개변수로 수행할 교차 검증 횟수: 9 x 15 x 10 = 1350번
# 기본적으로 5-폴드 교차 검증 수행: 1350 x 5 = 6770
# n_jobs 매개변수 -1로 설정 후 그리드 서치 실행
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [19]:
# 최상의 매개변수 조합 결과
print(gs.best_params_)
# 6770개 모델로 교차 검증 결과가 가장 좋은 결과를 내는 매개변수 값

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [20]:
# 최상의 교차 검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


# 랜덤 서치
- GridSearch에서 매개변수의 간격을 0.0001 혹은 1로 미리 설정
- 매개변수의 값이 수치일 때 범위나 간격을 미리 정하기 어려움
- 이럴 때 랜덤 서치(Random Search) 사용
- 매개변수 값 목록을 전달하는 것이 아니라 -> 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달


In [21]:
# 싸이파이 에서 2개의 확률 분포 클래스 임포트
# 파이썬의 핵심 과학 라이브러리 중 하나
# 적분, 보간, 선형 대수, 확률 등을 포함한 수치 계산 전용 라이브러리
# 사이킷런은 넘파이와 싸이파아ㅣ 기능을 많이 사용
from scipy.stats import uniform, randint

In [23]:
# uniform. randint 클래스는 모두 주어진 범위에서 고르게 값을 뽑음 -> rbsemd qnsvhdptj toavmffldgksek
# randint: 정수값을 뽑음
# uniform: 실수값을 뽑음
rgen = randint(0, 10) # 0 ~ 10으로 난수(정수)의 범위를 지정 
rgen.rvs(20) # 난수 20개 추출

array([7, 5, 3, 9, 3, 1, 5, 8, 7, 3, 5, 7, 5, 7, 9, 1, 2, 8, 9, 5])

In [24]:
ugen = uniform(0, 1) # 0 ~ 1으로 난수(실수)의 범위를 지정
ugen.rvs(20) # 난수 20개 추출

array([0.52649091, 0.08640886, 0.63366613, 0.51027587, 0.72151488,
       0.36647411, 0.74650872, 0.39213643, 0.77789946, 0.92566881,
       0.54760621, 0.71002111, 0.41490416, 0.75212662, 0.80027391,
       0.35297312, 0.81359644, 0.72325469, 0.43685975, 0.80175162])

- `min_samples_leaf` 매개변수 추가
  - 리프 노드가 되기 위한 최소 샘플의 개수
  - 어떤 노드가 분할하여 마들어진 자식 노드의 샘플 수가 이 값보다 작을 경우 분할하지 않음

In [25]:
params = {
    'min_impurity_decrease': uniform(0.0001, 0.001),
    'max_depth': randint(20, 50),
    'min_samples_split': randint(2, 25),
    'min_samples_leaf': randint(1, 25)
}

- `n_iter` 매개변수: 샘플링 횟수
- params에 정의된 매개변수 범위에서 총 100번 샘플링하여 교차 검증
- 그리드 서치보다 교차 검증 수를 줄이지만, 넓은 영역을 효과적으로 탐색

In [26]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42,), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f6a4e5b4550>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f6a4e55c810>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f6a4e7a0f90>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f6a4e5b4110>},
                   random_state=42)

In [27]:
# 최적의 매개변수 조합
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [28]:
# 최고의 교차 검증 점수
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [29]:
# 최적의 모델은 best_estimator_ 속성에 저장
# 이를 최종 모델로 결정하고 테스트 세트의 성능 확인
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
